In [1]:
from sklearn.metrics import jaccard_score
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import spacy
import re

In [2]:
df_timestamps = pd.read_csv('with_timestamps.csv', index_col=0)
df_who_said = pd.read_csv('who_said_splitted.csv', index_col=0)

print(df_timestamps.head(2))
print('='*80)
print(df_who_said.head(2))
print('='*25)
print(df_timestamps.shape, df_who_said.shape)

                                        line         start           end
0       Report to your stations immediately.  00:00:42,800  00:00:44,802
1  This is not a drill. We are under attack!  00:00:44,960  00:00:47,721
     speaker                                 line
0  Announcer  Report to your stations immediately
1  Announcer                  This is not a drill
(1893, 3) (1847, 2)


In [3]:
nlp = spacy.load("en_core_web_md")

# zasięg indeksów, w których szukamy podobnych zdań -> osobę, która je wypowiedziała
comparison_range = 20
for i, row in df_timestamps.iterrows():
    if i > 600:
        comparison_range = 30

    time_line = nlp(row['line'])
    highest_similarity = 0
    bottom_limit = i-comparison_range if i-comparison_range >= 0 else i

    for j in range(bottom_limit, min(i + 1 + comparison_range, len(df_who_said))):
        compared_row = df_who_said.loc[j]
        compared_line = nlp(str(compared_row['line']))
        similarity = time_line.similarity(compared_line)

        if similarity >= highest_similarity:
            df_timestamps.at[i, 'who_said'] = df_who_said.iloc[j]['speaker']
            df_timestamps.at[i, 'similarity'] = np.round(similarity, 2)
            df_timestamps.at[i, 'similar_line'] = compared_row['line']
            highest_similarity = similarity

C:\Users\Patryk\AppData\Local\Temp\ipykernel_11392\751405105.py:16: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = time_line.similarity(compared_line)


In [ ]:
df = df_timestamps[['start', 'end', 'who_said', 'line', 'similarity', 'similar_line']]

In [ ]:
df.to_csv('merged.csv')